Goal here is to build a basic model, reiterate later upon study of other solutions, revisit with different valuation method

**Rough draft to be edited for clarity**

In NFL, what is the best way to distribute value across cap?

Clustering (assumptions): minimum / rookie contract / free agent / franchise tag / outliers

53+ roster spots.

How many of each contract can be afforded, and what is the optimal distribution?

Then account for positional value?


cluster by salary, Measure AV/salary for all and determine optimal distribution of salaries by contract type by AV (as a basic measure of player value)

**Import the data**

may need to use API's &/or webscraping. 

Use footballref and spotrac as the sources?

**Inspect dataset, prepare data appropriately**

Want to have dataframe that at least has player position({'str':float64}), salary(real()), value(av()), and yearly salary cap (real()) formatted properly

**Cluster salaries**

I think this is mostly done using an ai?

**Evaluate all players value compared to salary**

value_per_dollar = av / salary

**Visualize and evaluate**
analyze by clusters (use mean. assumpt: In NFL, team control to block player movement prevents teams from paying above market rate, and that teams would do so to extract value. However, they can't so much of how it is distributed is (assumed)random. so each team is as likely to acquire players anywhere within that cluster of contracts.

Use salary earned rather than cap figure to account for fact that that number fluctuates to accomodate teams to their cap needs, but doesn't reflect cost as effectively as salary)

perc_cap = salary / salary_cap 

order clusters by value, how many of each can exist on a team, best distribution given need for at least 53(how many could become better understood)

**Ideas to explore**

better ways to evaluate value than av

how many players are needed on a roster, does that affect distribution?

evaluate by positional value (did it have underlying effect in previous inquiry?)

find teams historically that fit similar construction based on first inquiry, and compare their relative performance (AV, W/L, super bowl, etc)/
find all possible clusters, look for their occurences in nfl teams and compare by their relative performance (AV, W/L, super bowl, etc)